<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [2]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [4]:
def x1(x, v):
  return math.sin(math.log(abs(v[0])) / x[0])

def x2(x, v):
  return math.cos(v[1]**3)

def x3(x, v):
  return math.sin(math.tan(x[0] / x[1]) - 3**(v[2]))

def v1(x, v):
  return (v[0] + math.tan(x[0])**2)

def v2(x, v):
  return (0.2 * v[1])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * v[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[1])))

def y3(x, v):
  return v[2] * x[0] / (x[1] + 0.1)

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [5]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [6]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_3 = createTableDNM(functions, count, 3, np.array(model_2))
model_4 = createTableDNM(functions, count, 4, np.array(model_3))
model_5 = createTableDNM(functions, count, 5, np.array(model_4))
model_6 = createTableDNM(functions, count, 6, np.array(model_5))
model_7 = createTableDNM(functions, count, 7, np.array(model_6))
model_8 = createTableDNM(functions, count, 8, np.array(model_7))
model_9 = createTableDNM(functions, count, 9, np.array(model_8))
model_10 = createTableDNM(functions, count, 10, np.array(model_9))

In [7]:
model_9

,x1(9),x2(9),x3(9),v1(9),v2(9),v3(9),y1(9),y2(9),y3(9)
0,-0.545241,0.999999,-0.841985,21.493816,-0.125941,0.796405,-0.018731,0.354412,-0.394757
1,0.904902,0.999989,-0.912804,0.767241,0.330135,12.234426,1.121941,0.569360,10.064599
2,0.945172,0.989788,0.880955,-1.924394,0.192965,12.439418,-0.477499,0.437915,10.788697
3,0.796715,0.999998,-0.366485,0.489583,-0.176694,2.074875,1.452540,0.419257,1.502806
4,-0.839011,1.000000,0.788883,-1.544209,0.421821,1.489227,0.497675,0.639861,-1.135890
...,...,...,...,...,...,...,...,...,...
95,-0.991195,0.987346,-0.995114,2.349405,-0.539046,0.929923,-0.420030,0.716462,-0.847692
96,-0.968698,0.985913,-0.002390,65.419970,0.141816,1.530739,-0.014574,0.375953,-1.365509
97,0.792645,0.999721,0.913831,2833.170022,0.544848,-0.633417,0.000249,0.719922,-0.456547
98,0.808622,0.998073,0.860562,-0.954012,0.130860,0.411222,-0.762192,0.361229,0.302824


In [8]:
model_10

,x1(10),x2(10),x3(10),v1(10),v2(10),v3(10),y1(10),y2(10),y3(10)
0,0.610544,0.999998,-0.135863,0.635283,-0.207189,0.012484,0.750946,0.453552,0.006929
1,-0.288633,0.999353,0.858695,5571.739652,0.028463,10.797363,-0.000028,0.168699,-2.834826
2,0.638528,0.999974,-0.902076,-2.331510,-0.481052,2.551449,-0.218843,0.680229,1.481100
3,-0.781105,0.999985,-0.625715,2.425516,0.481571,0.510511,-0.284616,0.680567,-0.362517
4,-0.495044,0.997185,0.034745,28.296754,0.202816,1.224729,-0.012309,0.448807,-0.552592
...,...,...,...,...,...,...,...,...,...
95,-0.758983,0.987758,0.934537,2.378887,-0.198107,1.017192,-0.277955,0.443637,-0.709746
96,0.922431,0.999996,-0.556870,1.541594,-0.170490,1.711450,0.574687,0.411905,1.435183
97,-0.567828,0.986948,0.493764,1.787968,-0.384430,-0.052639,-0.239313,0.612397,0.027499
98,-0.058188,0.999997,-0.497102,1.260909,-0.314289,12.408085,-0.011132,0.556004,-0.656364


##1. Линейная форма функции пересчёта

In [9]:
def buildLinearRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  coef = []

  linearFunction = (model1.copy())[cols[0:(len(cols) - 1)]]
  linearFunction[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]
  linearFunction.insert(0, "optional_col", vector_ones)

  A_linear = np.array(linearFunction)
  #A_psev = BenIsrael(A_linear[:, 0:(len(A_linear[0]) - 1)])
  X_linear = solvingSLAE(A_linear[:, 0:(len(A_linear[0]) - 1)], np.matrix(A_linear[:, len(A_linear[0]) - 1]).T)

  estimate_linear = []
  variance_linear = []
  error_linear = []

  for i in range(len(linearFunction)):
    for j in range(0, len(X_linear)):
      if(j == 0):
        estimate_linear.append(X_linear[j,0])
      else:
        estimate_linear[i] += X_linear[j,0] * np.array(linearFunction)[i,j]

    variance_linear.append((1.0 / len(linearFunction)) * (np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])**2)
    #error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            #/ np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1]) * 100)
    error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            / max(np.array(linearFunction)[:, len(np.array(linearFunction)[0]) - 1])) * 100)

  linearFunction['regression'] = estimate_linear
  linearFunction['variance'] = variance_linear
  linearFunction['error, %'] = error_linear

  return X_linear, linearFunction

###Для функции $x_1[t+1]$

In [10]:
#x1, v1+
X1_linear, X1_linearTable = buildLinearRegretion(model_9, model_10, ['x1(9)', 'v1(9)', 'x1(10)'])
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c11', 'b11']))
X1_linearTable

           X1
a1  -0.006366
c11  0.335646
b11 -0.000184


,optional_col,x1(9),v1(9),x1(10),regression,variance,"error, %"
0,1,-0.545241,21.493816,0.610544,-0.193333,0.006462,0.811567
1,1,0.904902,0.767241,-0.288633,0.297220,0.003432,0.591457
2,1,0.945172,-1.924394,0.638528,0.311232,0.001071,0.330427
3,1,0.796715,0.489583,-0.781105,0.260958,0.010859,1.052032
4,1,-0.839011,-1.544209,-0.495044,-0.287692,0.000430,0.209335
...,...,...,...,...,...,...,...
95,1,-0.991195,2.349405,-0.758983,-0.339489,0.001760,0.423507
96,1,-0.968698,65.419970,0.922431,-0.343555,0.016027,1.278097
97,1,0.792645,2833.170022,-0.567828,-0.262139,0.000934,0.308614
98,1,0.808622,-0.954012,-0.058188,0.265221,0.001046,0.326502


In [11]:
print("E = " + str(sum(X1_linearTable['variance'])))
print("A = " + str(sum(X1_linearTable['error, %'])))

E = 0.36403701616699424
A = 53.703421033136365


###Для функции $x_2[t+1]$

In [12]:
#v2+
X2_linear, X2_linearTable = buildLinearRegretion(model_9, model_10, ['v2(9)', 'x2(10)'])
print(pd.DataFrame(X2_linear, columns=['X2'], index=['a2', 'b22']))
X2_linearTable

           X2
a2   0.996278
b22  0.002756


,optional_col,v2(9),x2(10),regression,variance,"error, %"
0,1,-0.125941,0.999998,0.995931,1.654235e-07,0.004067
1,1,0.330135,0.999353,0.997188,4.687272e-08,0.002165
2,1,0.192965,0.999974,0.996810,1.001403e-07,0.003164
3,1,-0.176694,0.999985,0.995791,1.758869e-07,0.004194
4,1,0.421821,0.997185,0.997440,6.542892e-10,0.000256
...,...,...,...,...,...,...
95,1,-0.539046,0.987758,0.994792,4.947398e-07,0.007034
96,1,0.141816,0.999996,0.996669,1.107033e-07,0.003327
97,1,0.544848,0.986948,0.997779,1.173210e-06,0.010831
98,1,0.130860,0.999997,0.996639,1.128263e-07,0.003359


In [13]:
print("E = " + str(sum(X2_linearTable['variance'])))
print("A = " + str(sum(X2_linearTable['error, %'])))

E = 2.90422039621223e-05
A = 0.42246845049471204


###Для функции $x_3[t+1]$

In [14]:
#x1, x2, v3+
X3_linear, X3_linearTable = buildLinearRegretion(model_9, model_10, ['x1(9)', 'x2(9)', 'v3(9)', 'x3(10)'])
print(pd.DataFrame(X3_linear, columns=['X3'], index=['a3', 'c31', 'c32', 'b33']))
X3_linearTable

            X3
a3   16.756786
c31   0.156396
c32 -16.879568
b33   0.012619


,optional_col,x1(9),x2(9),v3(9),x3(10),regression,variance,"error, %"
0,1,-0.545241,0.999999,0.796405,-0.135863,-0.197994,0.000039,0.062132
1,1,0.904902,0.999989,12.234426,0.858695,0.173304,0.004698,0.685395
2,1,0.945172,0.989788,12.439418,-0.902076,0.354384,0.015787,1.256468
3,1,0.796715,0.999998,2.074875,-0.625715,0.028032,0.004274,0.653751
4,1,-0.839011,1.000000,1.489227,0.034745,-0.235205,0.000729,0.269952
...,...,...,...,...,...,...,...,...
95,1,-0.991195,0.987346,0.929923,0.934537,-0.052469,0.009742,0.987012
96,1,-0.968698,0.985913,1.530739,-0.556870,-0.017192,0.002913,0.539681
97,1,0.792645,0.999721,-0.633417,0.493764,-0.002095,0.002459,0.495863
98,1,0.808622,0.998073,0.411222,-0.497102,0.041391,0.002900,0.538496


In [15]:
print("E = " + str(sum(X3_linearTable['variance'])))
print("A = " + str(sum(X3_linearTable['error, %'])))

E = 0.47391909080443817
A = 60.8256635659305


###Для функции $y_1[t]$

In [16]:
#x1, v1+
Y1_linear, Y1_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'v1(9)', 'y1(9)'])
print(pd.DataFrame(Y1_linear, columns=['Y1'], index=['a1', 'c11', 'b11']))
Y1_linearTable

           Y1
a1   0.009487
c11  0.240102
b11 -0.000007


,optional_col,x1(9),v1(9),y1(9),regression,variance,"error, %"
0,1,-0.545241,21.493816,-0.018731,-0.121571,0.000106,0.025425
1,1,0.904902,0.767241,1.121941,0.226750,0.008014,0.221320
2,1,0.945172,-1.924394,-0.477499,0.236437,0.005097,0.176508
3,1,0.796715,0.489583,1.452540,0.200776,0.015669,0.309477
4,1,-0.839011,-1.544209,0.497675,-0.191951,0.004756,0.170498
...,...,...,...,...,...,...,...
95,1,-0.991195,2.349405,-0.420030,-0.228516,0.000367,0.047348
96,1,-0.968698,65.419970,-0.014574,-0.223538,0.000437,0.051663
97,1,0.792645,2833.170022,0.000249,0.180822,0.000326,0.044643
98,1,0.808622,-0.954012,-0.762192,0.203645,0.009328,0.238786


In [17]:
print("E = " + str(sum(Y1_linearTable['variance'])))
print("A = " + str(sum(Y1_linearTable['error, %'])))

E = 0.42366303158138413
A = 9.017423294269753


###Для функции $y_2[t]$

In [18]:
#v2+
Y2_linear, Y2_linearTable = buildLinearRegretion(model_9, model_9, ['v2(9)', 'y2(9)'])
print(pd.DataFrame(Y2_linear, columns=['Y2'], index=['a2', 'b22']))
Y2_linearTable

           Y2
a2   0.526959
b22 -0.039026


,optional_col,v2(9),y2(9),regression,variance,"error, %"
0,1,-0.125941,0.354412,0.531874,0.000315,0.236832
1,1,0.330135,0.569360,0.514075,0.000031,0.073781
2,1,0.192965,0.437915,0.519428,0.000066,0.108784
3,1,-0.176694,0.419257,0.533854,0.000131,0.152937
4,1,0.421821,0.639861,0.510497,0.000167,0.172644
...,...,...,...,...,...,...
95,1,-0.539046,0.716462,0.547996,0.000284,0.224827
96,1,0.141816,0.375953,0.521424,0.000212,0.194139
97,1,0.544848,0.719922,0.505695,0.000459,0.285898
98,1,0.130860,0.361229,0.521852,0.000258,0.214360


In [19]:
print("E = " + str(sum(Y2_linearTable['variance'])))
print("A = " + str(sum(Y2_linearTable['error, %'])))

E = 0.03018052241694537
A = 19.808640125382606


###Для функции $y_3[t]$

In [21]:
#x1, x2, v3+
Y3_linear, Y3_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'x2(9)', 'v3(9)', 'y3(9)'])
print(pd.DataFrame(Y3_linear, columns=['Y3'], index=['a3', 'c31', 'c32', 'b33']))
Y3_linearTable

            Y3
a3   19.294075
c31   2.917772
c32 -19.611830
b33   0.041303


,optional_col,x1(9),x2(9),v3(9),y3(9),regression,variance,"error, %"
0,1,-0.545241,0.999999,0.796405,-0.394757,-1.875736,0.021933,0.078254
1,1,0.904902,0.999989,12.234426,10.064599,2.828070,0.523674,0.382371
2,1,0.945172,0.989788,12.439418,10.788697,3.154105,0.582870,0.403405
3,1,0.796715,0.999998,2.074875,1.502806,2.092610,0.003479,0.031165
4,1,-0.839011,1.000000,1.489227,-1.135890,-2.704285,0.024599,0.082873
...,...,...,...,...,...,...,...,...
95,1,-0.991195,0.987346,0.929923,-0.847692,-2.923253,0.043080,0.109671
96,1,-0.968698,0.985913,1.530739,-1.365509,-2.804708,0.020713,0.076046
97,1,0.792645,0.999721,-0.633417,-0.456547,1.974317,0.059091,0.128445
98,1,0.808622,0.998073,0.411222,0.302824,2.096389,0.032169,0.094770


In [22]:
print("E = " + str(sum(Y3_linearTable['variance'])))
print("A = " + str(sum(Y3_linearTable['error, %'])))

E = 11.203257053239
A = 11.212760133558831


##2. Квадратичная форма функции пересчёта